# Regression B score...

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from jupyterthemes import jtplot
jtplot.style(theme='onedork', context='notebook', ticks=True, grid=False)

In [ ]:
import pandas as pd
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
pd.set_option("display.max_columns", None)
import numpy as np
import os
import matplotlib.pyplot as plt
from pylab import rcParams
rcParams['figure.figsize'] = 15, 5

import data
import copy
from rolldecay.bis_system import BisSystem

from rolldecay import database
from mdldb.tables import Run
from rolldecayestimators.direct_estimator import DirectEstimator
from rolldecayestimators.ikeda_estimator import IkedaQuadraticEstimator
from rolldecayestimators import symbols, equations
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from rolldecayestimators.transformers import CutTransformer, LowpassFilterDerivatorTransformer, ScaleFactorTransformer, OffsetTransformer


import sympy as sp

from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import VarianceThreshold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.feature_selection import f_regression

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.metrics import r2_score

import signal_lab
from sqlalchemy.inspection import inspect
import seaborn as sns
import docs
import pickle
import gc
import dill
from rolldecayestimators.substitute_dynamic_symbols import lambdify
from rolldecayestimators.polynom_estimator import Polynom
from scipy.integrate import trapz

## omega0

In [ ]:
filename = 'omega0_hat_model.sav'
omega0_hat_model = pickle.load(open(filename, 'rb'))
data = pd.read_pickle('data.sav')
y_s = pd.read_pickle('y.sav')

In [ ]:
y = y_s['omega0_hat']
X=data[omega0_hat_model.keys]
omega0_hat_model.score(X=X,y=y)

In [ ]:
omega_hat_regression_polynom = Polynom.load('omega0_hat_polynom.sym')
omega_hat_regression_polynom.score(X=X,y=y)

## zeta

In [ ]:
filename = 'zeta_model.sav'
zeta_model = pickle.load(open(filename, 'rb'))

In [ ]:
y = y_s['zeta']
#X=data[zeta_model.keys].copy()  # Handle categorical data later
#zeta_model.score(X=X,y=y)

In [ ]:
zeta_regression_polynom = Polynom.load('zeta_polynom.sym')
zeta_regression_polynom.score(X=data,y=y)

## d

In [ ]:
filename = 'd_model.sav'
d_model = pickle.load(open(filename, 'rb'))

In [ ]:
X=data[d_model.keys].copy()  # Handle categorical data later

In [ ]:
y = y_s['d']
d_model.score(X=X,y=y)

In [ ]:
d_regression_polynom = Polynom.load('d_polynom.sym')
d_regression_polynom.score(X=data,y=y)

In [ ]:
X.shape

## Predict parameters

In [ ]:
predict = data.copy()
#predict['omega_hat'] = omega0_hat_model.predict(data[omega0_hat_model.keys])
predict['omega_hat'] = omega_hat_regression_polynom.predict(data)

#predict['zeta'] = zeta_model.predict(data[zeta_model.keys])
predict['zeta'] = zeta_regression_polynom.predict(data)

#predict['d'] = d_model.predict(data[d_model.keys])
predict['d'] = d_regression_polynom.predict(data)


## Simulate

In [ ]:
df_rolldecay = database.load(rolldecay_table_name='rolldecay_direct', limit_score=0.90)
db = database.get_db()

In [ ]:
df_rolldecay_ikeda = database.load(rolldecay_table_name='rolldecay_simplified_ikeda', limit_score=0.0)

In [ ]:
predict.head()

In [ ]:
run = predict.iloc[0]

In [ ]:
meta_data_full_scale = df_rolldecay.loc[run.name]

In [ ]:
db_run = db.session.query(Run).get(int(run.name))
df = database.load_run(db_run)

In [ ]:
df.plot(y='phi')

In [ ]:
lowpass_filter = LowpassFilterDerivatorTransformer(cutoff=2, minimum_score=0.99)
scaler = ScaleFactorTransformer(scale_factor=db_run.model.scale_factor)  # dummy value None for now
cutter = CutTransformer(phi_max=np.deg2rad(9), phi_min=np.deg2rad(0.25))
offset_transformer = OffsetTransformer()

steps = [
    ('filter',lowpass_filter),
    ('scaler',scaler),  # Is froude scaling a good idea??
    ('cutter', cutter), 
    ('offset_transformer',offset_transformer)
]
        
preprocessor = Pipeline(steps) # define the pipeline object.
preprocessor.fit(df)
X = preprocessor.transform(df)

In [ ]:
X.plot(y='phi')

In [ ]:
omega_hat_lambda=lambdify(sp.solve(equations.omega_hat_equation,symbols.omega)[0])

In [ ]:
g=9.81
omega0=omega_hat_lambda(beam=meta_data_full_scale.beam, g=g, omega_hat=run.omega_hat)

In [ ]:
run_data = run.copy()
run_data['omega0']=omega0

In [ ]:
direct_estimator = DirectEstimator.load(data=run_data, X=X)

In [ ]:
direct_estimator.plot_fit()

In [ ]:
direct_estimator.plot_error()

### Using "real" omega0

In [ ]:
run_data = run.copy()
run_data['omega0']=meta_data_full_scale.omega0

In [ ]:
direct_estimator = DirectEstimator.load(data=run_data, X=X)

In [ ]:
fig,ax=plt.subplots()
direct_estimator.plot_fit(ax=ax)

meta_data_ikeda = df_rolldecay_ikeda.loc[run.name]
meta_data_ikeda['omega0']=meta_data_full_scale.omega0
ikeda_estimator = IkedaQuadraticEstimator.load(data=meta_data_ikeda, X=X)
ikeda_estimator.plot_fit(ax=ax, model_test=False)

In [ ]:
direct_estimator.score()

In [ ]:
ikeda_estimator.score()

In [ ]:
B1_lambda = lambdify(sp.solve(equations.zeta_B1_equation,symbols.B_1)[0])
g=9.81
rho=1000
m=meta_data_full_scale['Volume']*rho

In [ ]:
B2_lambda = lambdify(sp.solve(equations.d_B2_equation,symbols.B_2)[0])

In [ ]:
B_e = lambdify(sp.solve(equations.B_e_equation, symbols.B_e)[0])
B_e

In [ ]:
def calculate_B_e(GM,g, m, omega0, zeta, d, phi_a):
    df = pd.DataFrame()
    df['phi_a']=phi_a
    df.set_index('phi_a', inplace=True)
    
    df['B_1'] = B1_lambda(GM=GM,g=g, m=m, omega0=omega0, 
          zeta=zeta)

    df['B_2'] = B2_lambda(GM=GM,g=g, m=m, omega0=omega0, 
          d=d)

    df['B_e'] = B_e(B_1=df['B_1'], B_2=df['B_2'], omega0=omega0, phi_a=phi_a)
    
    return df

In [ ]:
phi_a = np.linspace(X['phi'].abs().min(),X['phi'].abs().max(),10)

omega0=meta_data_full_scale.omega0


df = calculate_B_e(GM=meta_data_full_scale['gm'],g=g, m=m, omega0=omega0, 
          zeta=meta_data_full_scale['zeta'], d=meta_data_full_scale['d'], phi_a=phi_a)

df_regression = calculate_B_e(GM=meta_data_full_scale['gm'],g=g, m=m, omega0=omega0, 
          zeta=direct_estimator.parameters['zeta'], d=direct_estimator.parameters['d'], phi_a=phi_a) 

df_ikeda = calculate_B_e(GM=meta_data_full_scale['gm'],g=g, m=m, omega0=omega0, 
          zeta=ikeda_estimator.parameters['zeta'], d=ikeda_estimator.parameters['d'], phi_a=phi_a) 

In [ ]:
fig,ax=plt.subplots()
df.plot(y='B_e',label='DB', ax=ax, style='k-')
df_regression.plot(y='B_e',label='regression', ax=ax)
df_ikeda.plot(y='B_e',label='ikeda', ax=ax)

ax.fill_between(df.index, df['B_e'], df_regression['B_e'], label='error regression',alpha=1)
ax.fill_between(df.index, df['B_e'], df_ikeda['B_e'], label='error ikeda', alpha=0.25)
ax.legend()

ax.set_title('Comparison of Equivalent damping vs. roll amplitude')
ax.set_ylabel('$B_e$ [Nm*s/rad] (Equivalent damping)')
ax.set_xlabel('$\phi_a$ [rad] (Roll amplitude)')



In [ ]:
r2_score(y_true=df['B_e'], y_pred=df_regression['B_e'])

In [ ]:
r2_score(y_true=df['B_e'], y_pred=df_ikeda['B_e'])

In [ ]:
(df['B_e']
 
 
 -df_regression['B_e']).abs()/df['B_e']

In [ ]:
X['phi'].abs().hist(bins=100)

In [ ]:
B_e_equation =sp.solve(equations.B_e_equation, symbols.B_e)[0]
B_e_equation

In [ ]:
phi_min, phi_max, Be_tot = sp.symbols('phi_min phi_max B_e_tot')

Be_integral = sp.Eq(Be_tot,sp.Integral(B_e_equation,(symbols.phi_a,phi_min,phi_max)))
Be_integral

In [ ]:
Be_tot_equation = sp.Eq(Be_tot,Be_integral.rhs.simplify())
Be_tot_equation

In [ ]:
Be_tot_lambda = lambdify(sp.solve(Be_tot_equation, Be_tot)[0])

In [ ]:
Be_tot = Be_tot_lambda(B_1=df.iloc[0]['B_1'], B_2=df.iloc[0]['B_2'], omega0=omega0, phi_max=np.max(phi_a), 
              phi_min=np.min(phi_a))

Be_tot_regression = Be_tot_lambda(B_1=df_regression.iloc[0]['B_1'], B_2=df_regression.iloc[0]['B_2'], 
                                  omega0=omega0, phi_max=np.max(phi_a),phi_min=np.min(phi_a))

Be_tot_ikeda = Be_tot_lambda(B_1=df_ikeda.iloc[0]['B_1'], B_2=df_ikeda.iloc[0]['B_2'], 
                                  omega0=omega0, phi_max=np.max(phi_a),phi_min=np.min(phi_a))



In [ ]:
def score(Be_tot_true, Be_tot_predicted):
    return 1-np.abs(Be_tot_true-Be_tot_predicted)/((Be_tot_true+Be_tot_predicted)/2)

In [ ]:
score(Be_tot_true=Be_tot, Be_tot_predicted=Be_tot_regression)

In [ ]:
score(Be_tot_true=Be_tot, Be_tot_predicted=Be_tot_ikeda)